# Basis rotation circuit

![title](img/Slater_determinant.png)


The unitary $U_k$ can be compiled exactly (without Trotterization) as a circuit which has gate depth $\frac{N}{2}$:

![title](img/basic_circuit.png)

## $\theta$ operation

![title](img/theta_operation.png)

In [3]:
from mindquantum import Circuit
from mindquantum.core import RZ, X, ISWAP, Power, apply, BARRIER
from mindquantum.core.parameterresolver import ParameterResolver
import math


def theta_operation(theta):
    # theta is a string.
    pr = ParameterResolver(theta)
    circuit = Circuit()
    circuit += Power(ISWAP, 0.5).on([0, 1])
    circuit += RZ(-pr).on(0)
    circuit += RZ(pr + math.pi).on(1)
    circuit += Power(ISWAP, 0.5).on([0, 1])
    circuit += RZ(math.pi).on(1)
    return circuit

def test_circ(theta):
    circ = Circuit()
    circ += X.on(0)
    circ += RZ(theta).on(1)
    return circ

circ = theta_operation('theta')
circ.svg()

finally, we can build the whole circuit:

In [4]:
from mindquantum.core import apply, X, RZ, BARRIER


def half_filling_ansatz_circuit(n_electrons, count):
    circuit = Circuit()
    for depth in range(n_electrons):
        n_ops = depth + 1  # the number of gate in each layer
        q = n_electrons - n_ops  # index of start qubit
        for op in range(n_ops):  # add n_ops theta_operation
            circuit += apply(theta_operation(f'theta_{count}'), [q, q+1])
            q += 2  # next theta_operation
            count += 1
        circuit += BARRIER

    for depth in range(n_electrons - 1):
        n_ops = n_electrons - depth - 1
        q = depth+1
        for op in range(n_ops):  # add n_ops theta_operation
            circuit += apply(theta_operation(f'theta_{count}'), [q, q+1])
            q += 2  # next theta_operation
            count += 1
        circuit += BARRIER
    return circuit


def non_half_filling_ansatz_circuit(n_qubits, n_electrons):
    circuit = Circuit()
    unoccupied_orbitals = n_qubits - n_electrons
    if n_electrons > unoccupied_orbitals:
        count = 1  # the number of theta_operation
        for depth in range(unoccupied_orbitals):
            n_ops = depth + 1  # the number of gate in each layer
            q = n_electrons - n_ops  # index of start qubit
            for op in range(n_ops):  # add n_ops theta_operation
                circuit += apply(theta_operation(f'theta_{count}'), [q, q + 1])
                q += 2  # next theta_operation
                count += 1
            circuit += BARRIER
        for depth in range(n_electrons - 1):
            n_ops = unoccupied_orbitals - depth - 1
            q = n_electrons - n_ops
            for op in range(n_ops):  # add n_ops theta_operation
                circuit += apply(theta_operation(f'theta_{count}'), [q, q + 1])
                q += 2  # next theta_operation
                count += 1
            circuit += BARRIER
        circuit += half_filling_ansatz_circuit(int(n_electrons/2), count)
    else:
        count = 1  # the number of theta_operation
        for depth in range(int(unoccupied_orbitals/2)):
            n_ops = depth + 1  # the number of gate in each layer
            q = n_electrons - n_ops + int(unoccupied_orbitals/2)  # index of start qubit
            for op in range(n_ops):  # add n_ops theta_operation
                circuit += apply(theta_operation(f'theta_{count}'), [q, q + 1])
                q += 2  # next theta_operation
                count += 1
            circuit += BARRIER
        for depth in range(int(unoccupied_orbitals/2) - 1):
            n_ops = int(unoccupied_orbitals/2) - 1 - depth
            q = int(unoccupied_orbitals/2) - n_ops + n_electrons
            for op in range(n_ops):  # add n_ops theta_operation
                circuit += apply(theta_operation(f'theta_{count}'), [q, q + 1])
                q += 2  # next theta_operation
                count += 1
            circuit += BARRIER
        circuit += half_filling_ansatz_circuit(n_electrons, count)
    return circuit


def ansatz_circuit(n_qubits, n_electrons):
    hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(n_electrons)])
    if n_electrons == int(n_qubits/2):
        brc_ansatz_circuit = half_filling_ansatz_circuit(n_electrons, count=1)
    else:
        brc_ansatz_circuit = non_half_filling_ansatz_circuit(n_qubits, n_electrons)
    return hartreefock_wfn_circuit + brc_ansatz_circuit
    

ansatz_circuit(8, 4).svg()

# Results

|      | n_qubits | n_electrons | n_params |                                            |
| ---- | -------- | ----------- | -------- | :----------------------------------------- |
| H4   | 8        | 4           | 16       | 从键长1.3开始才有优化                      |
| LiH  | 12       | 4           | 32       | 几乎没有优化                               |
| BeH2 | 14       | 6           | 52       | 从键长2.4开始才有优化                      |
| H2O  | 14       | 10          | 41       | 从键长1.5开始才有优化（1.7跑了两个多小时） |
| CH4  | 18       | 10          | 89       |                                            |
| N2   | 20       | 14          | 85       |                                            |

# Measurement

## 1.  post-selection on particle number (total occupation number) when measuring all elements of the 1-RDM
reference: Appendix C: Optimal Measurement of the 1-RDM

### （1）Diagonal terms of the 1-RDM 

![title](img/measure_diagonal_terms.png)

Using the Jordan-Wigner transform to relate fermionic ladder operators $〈a^{†}_i a_i〉$, measurement result $M_i$ from qubit $i$, qubitoperators: 

![title](img/measure_diagonal_terms_eq.png)


### （2）One-off-diagonal terms of the 1-RDM 

![title](img/measure_one_off_diagonal_terms.png)

Using the Jordan-Wigner transform to map fermionic ladder operators to qubits:

![title](img/measure_one_off_diagonal_terms_eq.png)

we must measure $XX$ on all adjacent pairs and $YY$ on all adjacent pairs. 

### （3）General off-diagonal terms and virtual swapping



### （4）Off-diagonal terms with post-selection
design  a  basis  rotation  circuit that commutes with the total number operator and diagonalizes the $\frac{1}{2}(XX+YY)$ Hamiltonian. 

The diagonal form and its hermitian are below:
![title](img/measure_off_diagonal_terms_Um.png)
i.e.  $$U_M = \frac{1}{2}(XX+YY),   U^{†}_M = \frac{1}{2}(Z_a \bigotimes I - I \bigotimes Z_{a+1})$$

Therefor, we can relate the Z expectation values, the transformed $XX+YY$ expectation values, fermionic ladder operators, and binary measurements {$Z_{a}, Z_{a+1}$} via:

![title](img/measure_off_diagonal_terms_eq.png)

The diagonal form means that after performing the basis rotation 

![title](img/measure_off_diagonal_terms.png)

we can measure in the computational basis  to obtain expectation values $\frac{1}{2}〈XX+YY〉$.

The measurement circuit for a six qubit problem:

![title](img/measure_off_diagonal_terms_circ.png)



## 2.  pure-state projection (McWeeny purification)
Using McWeeny purification to perform projection back to the pure-set of 1-RDMs.

reference: Appendix E: Error mitigation through purification